#### Install required packages

In [1]:
!pip install requests
!pip install matplotlib
!pip install sklearn
!pip install geopy
!pip install folium
!pip install bs4

# Question 1: Capturing the Wikipedia Table

In [2]:
from IPython.display import HTML
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [3]:
html = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

In [4]:
soup = BeautifulSoup(html, 'html.parser')

In [5]:
Toronto_df = pd.DataFrame(columns=["Postal Code", "Borough", "Neighborhood"])
for row in soup.find("tbody").find_all("tr"):
    col = (row.find_all("td"))
    if(len(col)>0):
        pc = col[0].text.rstrip('\n')
        bh = col[1].text.rstrip('\n')
        nd = col[2].text.rstrip('\n')
        
        Toronto_df = Toronto_df.append({"Postal Code":pc,"Borough":bh,"Neighborhood":nd  }, ignore_index=True)
        
Toronto_df = Toronto_df[Toronto_df['Borough']!='Not assigned'].reset_index(drop=True)

In [6]:
Toronto_df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [7]:
toronto_grouped = Toronto_df.groupby(['Postal Code', 'Borough'], as_index=False)
toronto_concat = toronto_grouped.agg(lambda x: ", ".join(x))
toronto_concat.head(30)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [8]:
toronto_concat.shape

(103, 3)

# Question 2: Attach lat/long to each postalcode

The Geospatial_Coordinates.csv has been uploaded to the github, so that it can be directly read into the notebook.

In [9]:
pc_coordinates = pd.read_csv("Geospatial_Coordinates.csv")

In [10]:
pc_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
toronto_postcode_data = pd.merge(toronto_concat, pc_coordinates, on = 'Postal Code' )
toronto_postcode_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [12]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_postcode_data['Borough'].unique()),
        toronto_postcode_data.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


## Use folium library to create a map of Toronto City.

### We first find the Coordinates of Toronto to center the map

In [13]:
from geopy.geocoders import Nominatim
import requests

import folium

address = 'Toronto, CA'

geolocator = Nominatim(user_agent="tn_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


## Create a map of Toronto with neighborhoods superimposed on top

In [14]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_postcode_data['Latitude'], toronto_postcode_data['Longitude'], toronto_postcode_data['Borough'], toronto_postcode_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Focus on one Borough
For simplicity, we will focus on just one Borough, Scarborough. So we will create a subset for Scarborough.

In [15]:
Scarborough_data = toronto_postcode_data[toronto_postcode_data['Borough'] == 'Scarborough'].reset_index(drop=True)
Scarborough_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [16]:
address = 'Scarborough, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.7729744, -79.2576479.


## Visualize Scarborough and the neighborhoods in it.

In [17]:
# create map of Manhattan using latitude and longitude values
map_Scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Scarborough_data['Latitude'], Scarborough_data['Longitude'], Scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Scarborough)  
    
map_Scarborough

# Foursquare Connection

In [18]:
# @hidden cell

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>''')


CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CLIENT_SECRET:

We will explore the first neighborhood in our dataframe, but first we get the neighborhood's name.




In [19]:
Scarborough_data.loc[0, 'Neighborhood']

'Malvern, Rouge'

Get the neighborhood's latitude and longitude values.

In [20]:
neighborhood_latitude = Scarborough_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Scarborough_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Scarborough_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern, Rouge are 43.8066863, -79.1943534.


Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.
First, let's create the GET request URL. Name your URL url.

In [21]:
LIMIT = 100 
radius = 500 

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()

From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [22]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [23]:


venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy’s,Fast Food Restaurant,43.807448,-79.199056


In [24]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


# 2. Explore Neighborhoods in Scarborough

Function to get nearby venues

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

This Function call repeat the same process to all the neighborhoods in Scarborough

In [26]:

scarborough_venues = getNearbyVenues(names=Scarborough_data['Neighborhood'],
                                   latitudes=Scarborough_data['Latitude'],
                                   longitudes=Scarborough_data['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge


In [27]:
print(scarborough_venues.shape)
scarborough_venues.head()

(86, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant


Number of venues  returned for each neighborhood

In [28]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,9,9,9,9,9,9
"Clarks Corners, Tam O'Shanter, Sullivan",14,14,14,14,14,14
"Cliffside, Cliffcrest, Scarborough Village West",2,2,2,2,2,2
"Dorset Park, Wexford Heights, Scarborough Town Centre",5,5,5,5,5,5
"Golden Mile, Clairlea, Oakridge",8,8,8,8,8,8
"Guildwood, Morningside, West Hill",8,8,8,8,8,8
"Kennedy Park, Ionview, East Birchmount Park",4,4,4,4,4,4


#### what are the unique categories?

In [29]:
print('There are {} unique venue categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 53 unique venue categories.


# 3. Analyze Each Neighborhood

In [30]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,Accessories Store,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,...,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Soccer Field,Supermarket,Thai Restaurant,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [31]:
scarborough_onehot.shape

(86, 54)

Next, lets group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [32]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,Accessories Store,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,...,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Soccer Field,Supermarket,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.250000,0.00,...,0.000000,0.000000,0.000,0.000,0.000000,0.00,0.00,0.000000,0.000000,0.000000
1,"Birch Cliff, Cliffside West",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,...,0.000000,0.000000,0.000,0.000,0.000000,0.25,0.00,0.000000,0.000000,0.000000
2,Cedarbrae,0.000000,0.0,0.111111,0.000000,0.111111,0.111111,0.0,0.000000,0.00,...,0.000000,0.000000,0.000,0.000,0.000000,0.00,0.00,0.000000,0.111111,0.000000
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.000000,0.0,0.000000,0.000000,0.000000,0.071429,0.0,0.000000,0.00,...,0.142857,0.000000,0.000,0.000,0.000000,0.00,0.00,0.000000,0.071429,0.000000
4,"Cliffside, Cliffcrest, Scarborough Village West",0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,...,0.000000,0.000000,0.000,0.000,0.000000,0.00,0.00,0.000000,0.000000,0.000000
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,...,0.000000,0.000000,0.000,0.000,0.000000,0.00,0.00,0.000000,0.000000,0.200000
6,"Golden Mile, Clairlea, Oakridge",0.000000,0.0,0.000000,0.000000,0.250000,0.000000,0.0,0.000000,0.25,...,0.000000,0.000000,0.000,0.000,0.000000,0.00,0.00,0.000000,0.000000,0.000000
7,"Guildwood, Morningside, West Hill",0.000000,0.0,0.000000,0.000000,0.000000,0.125000,0.0,0.125000,0.00,...,0.000000,0.000000,0.125,0.125,0.000000,0.00,0.00,0.000000,0.000000,0.000000
8,"Kennedy Park, Ionview, East Birchmount Park",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,...,0.000000,0.000000,0.000,0.000,0.000000,0.00,0.00,0.000000,0.000000,0.000000
9,"Malvern, Rouge",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,...,0.000000,0.000000,0.000,0.000,0.000000,0.00,0.00,0.000000,0.000000,0.000000


Let's confirm the new size

In [33]:
scarborough_grouped.shape

(16, 54)

#### print each neighborhood with its top 5 most common venues

In [34]:
num_top_venues = 5

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Clothing Store  0.25
1  Latin American Restaurant  0.25
2                     Lounge  0.25
3             Breakfast Spot  0.25
4                  Pet Store  0.00


----Birch Cliff, Cliffside West----
                   venue  freq
0        College Stadium  0.25
1  General Entertainment  0.25
2           Skating Rink  0.25
3                   Café  0.25
4      Accessories Store  0.00


----Cedarbrae----
                venue  freq
0         Gas Station  0.11
1  Athletics & Sports  0.11
2     Thai Restaurant  0.11
3              Bakery  0.11
4                Bank  0.11


----Clarks Corners, Tam O'Shanter, Sullivan----
                  venue  freq
0              Pharmacy  0.14
1           Pizza Place  0.14
2  Fast Food Restaurant  0.14
3   Fried Chicken Joint  0.07
4           Gas Station  0.07


----Cliffside, Cliffcrest, Scarborough Village West----
                 venue  freq
0  American Restaurant   0.5
1             

#### We construct this into a  _pandas_ dataframe

First, let's write a function to sort the venues in descending order.


In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

And then create the new dataframe and display the top 10 venues for each neighborhood.


In [36]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Clothing Store,Latin American Restaurant,Lounge,Breakfast Spot,Pet Store,Italian Restaurant,Korean BBQ Restaurant,Medical Center,Metro Station,Mexican Restaurant
1,"Birch Cliff, Cliffside West",College Stadium,General Entertainment,Skating Rink,Café,Accessories Store,Italian Restaurant,Korean BBQ Restaurant,Latin American Restaurant,Lounge,Medical Center
2,Cedarbrae,Gas Station,Athletics & Sports,Thai Restaurant,Bakery,Bank,Lounge,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Accessories Store
3,"Clarks Corners, Tam O'Shanter, Sullivan",Pharmacy,Pizza Place,Fast Food Restaurant,Fried Chicken Joint,Gas Station,Convenience Store,Italian Restaurant,Chinese Restaurant,Noodle House,Bank
4,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Motel,Noodle House,Indian Restaurant,Intersection,Italian Restaurant,Korean BBQ Restaurant,Latin American Restaurant,Lounge,Medical Center


## 4. Cluster Neighborhoods

In [37]:
# set number of clusters
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 3])

We create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [38]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Label', kmeans.labels_)

scarborough_merged = Scarborough_data

# merge toronto_grouped with manhattan_data to add latitude/longitude for each neighborhood
scarborough_merged = (scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood'))


scarborough_merged_data=scarborough_merged.dropna() # check the last columns!

### Visualize the resulting clusters on a map

In [39]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged_data['Latitude'], scarborough_merged_data['Longitude'], scarborough_merged_data['Neighborhood'], scarborough_merged_data['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        color=rainbow[int(cluster)-1],
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine the resulting Clusters


### Cluster 1

In [40]:
scarborough_merged_data.loc[scarborough_merged_data['Cluster Label'] == 0, scarborough_merged_data.columns[[1] + list(range(5, scarborough_merged_data.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,0.0,Intersection,Bank,Breakfast Spot,Medical Center,Restaurant,Rental Car Location,Electronics Store,Mexican Restaurant,Noodle House,Italian Restaurant
4,Scarborough,0.0,Gas Station,Athletics & Sports,Thai Restaurant,Bakery,Bank,Lounge,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Accessories Store
6,Scarborough,0.0,Hobby Shop,Department Store,Discount Store,Coffee Shop,Soccer Field,Skating Rink,Italian Restaurant,Korean BBQ Restaurant,Latin American Restaurant,Lounge
7,Scarborough,0.0,Bakery,Bus Line,Ice Cream Shop,Bus Station,Metro Station,Intersection,Park,Italian Restaurant,Korean BBQ Restaurant,Latin American Restaurant
8,Scarborough,0.0,American Restaurant,Motel,Noodle House,Indian Restaurant,Intersection,Italian Restaurant,Korean BBQ Restaurant,Latin American Restaurant,Lounge,Medical Center
9,Scarborough,0.0,College Stadium,General Entertainment,Skating Rink,Café,Accessories Store,Italian Restaurant,Korean BBQ Restaurant,Latin American Restaurant,Lounge,Medical Center
10,Scarborough,0.0,Indian Restaurant,Chinese Restaurant,Vietnamese Restaurant,Pet Store,Soccer Field,Intersection,Italian Restaurant,Korean BBQ Restaurant,Latin American Restaurant,Lounge
11,Scarborough,0.0,Accessories Store,Bakery,Sandwich Place,Middle Eastern Restaurant,Vietnamese Restaurant,Auto Garage,Bank,Bar,Korean BBQ Restaurant,Latin American Restaurant
12,Scarborough,0.0,Clothing Store,Latin American Restaurant,Lounge,Breakfast Spot,Pet Store,Italian Restaurant,Korean BBQ Restaurant,Medical Center,Metro Station,Mexican Restaurant
13,Scarborough,0.0,Pharmacy,Pizza Place,Fast Food Restaurant,Fried Chicken Joint,Gas Station,Convenience Store,Italian Restaurant,Chinese Restaurant,Noodle House,Bank


### Cluster 2

In [41]:
scarborough_merged_data.loc[scarborough_merged_data['Cluster Label'] == 1, scarborough_merged_data.columns[[1] + list(range(5, scarborough_merged_data.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,1.0,Bar,Accessories Store,Park,Intersection,Italian Restaurant,Korean BBQ Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station


### Cluster 3

In [42]:
scarborough_merged_data.loc[scarborough_merged_data['Cluster Label'] == 2, scarborough_merged_data.columns[[1] + list(range(5, scarborough_merged_data.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,2.0,Playground,Cosmetics Shop,Accessories Store,Noodle House,Intersection,Italian Restaurant,Korean BBQ Restaurant,Latin American Restaurant,Lounge,Medical Center


### Cluster 4

In [43]:
scarborough_merged_data.loc[scarborough_merged_data['Cluster Label'] == 3, scarborough_merged_data.columns[[1] + list(range(5, scarborough_merged_data.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,3.0,Fast Food Restaurant,Accessories Store,Noodle House,Intersection,Italian Restaurant,Korean BBQ Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station


### Cluster 5

In [45]:

scarborough_merged_data.loc[scarborough_merged_data['Cluster Label'] == 4, scarborough_merged_data.columns[[1] + list(range(5, scarborough_merged_data.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,4.0,Coffee Shop,Soccer Field,Korean BBQ Restaurant,Accessories Store,Intersection,Italian Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station
